In [ ]:
import cv2
import numpy as np
import os
import pickle
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import face_recognition
from datetime import datetime
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time

class FaceRecognitionSystem:
    def __init__(self, model_path="models/", data_path="data/"):
        """
        Initialize Face Recognition System
        """
        self.model_path = model_path
        self.data_path = data_path
        self.create_directories()

        # Model components
        self.svm_model = None
        self.label_encoder = LabelEncoder()
        self.face_encodings = []
        self.face_labels = []

        # Recognition parameters
        self.recognition_threshold = 0.6
        self.confidence_threshold = 0.5

        # Statistics tracking
        self.recognition_stats = defaultdict(int)
        self.session_stats = {
            'total_detections': 0,
            'successful_recognitions': 0,
            'unknown_faces': 0,
            'start_time': datetime.now()
        }

    def create_directories(self):
        """Create necessary directories"""
        os.makedirs(self.model_path, exist_ok=True)
        os.makedirs(self.data_path, exist_ok=True)
        os.makedirs(f"{self.data_path}/faces", exist_ok=True)
        os.makedirs(f"{self.data_path}/unknown", exist_ok=True)

    def capture_training_data(self, person_name, num_images=50):
        """
        Capture training images for a person using webcam
        """
        print(f"Starting data collection for {person_name}")
        print("Press SPACE to capture image, 'q' to quit early")

        # Create person directory
        person_dir = f"{self.data_path}/faces/{person_name}"
        os.makedirs(person_dir, exist_ok=True)

        cap = cv2.VideoCapture(0)
        captured_count = 0

        while captured_count < num_images:
            ret, frame = cap.read()
            if not ret:
                break

            # Flip frame horizontally for natural interaction
            frame = cv2.flip(frame, 1)

            # Display progress
            cv2.putText(frame, f"Capturing for: {person_name}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Images captured: {captured_count}/{num_images}", (10, 70),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, "Press SPACE to capture, 'q' to quit", (10, 110),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # Detect faces in the frame
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)

            # Draw rectangles around faces
            for (top, right, bottom, left) in face_locations:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            cv2.imshow('Face Data Collection', frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord(' ') and len(face_locations) > 0:
                # Save the frame
                filename = f"{person_dir}/{person_name}_{captured_count:03d}.jpg"
                cv2.imwrite(filename, frame)
                captured_count += 1
                print(f"Captured image {captured_count}/{num_images}")

            elif key == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        print(f"Data collection completed for {person_name}: {captured_count} images")
        return captured_count

    def extract_face_encodings(self):
        """
        Extract face encodings from all training images
        """
        print("Extracting face encodings from training data...")

        self.face_encodings = []
        self.face_labels = []

        faces_dir = f"{self.data_path}/faces"

        for person_name in os.listdir(faces_dir):
            person_path = os.path.join(faces_dir, person_name)
            if not os.path.isdir(person_path):
                continue

            print(f"Processing {person_name}...")
            person_encodings = []

            for image_file in os.listdir(person_path):
                if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                image_path = os.path.join(person_path, image_file)

                # Load image
                image = face_recognition.load_image_file(image_path)

                # Get face encodings
                encodings = face_recognition.face_encodings(image)

                if encodings:
                    self.face_encodings.append(encodings[0])
                    self.face_labels.append(person_name)
                    person_encodings.append(encodings[0])

            print(f"Extracted {len(person_encodings)} encodings for {person_name}")

        print(f"Total encodings extracted: {len(self.face_encodings)}")
        return len(self.face_encodings)

    def train_model(self):
        """
        Train SVM model on extracted face encodings
        """
        if len(self.face_encodings) == 0:
            print("No face encodings available. Please extract encodings first.")
            return False

        print("Training SVM model...")

        # Convert to numpy arrays
        X = np.array(self.face_encodings)
        y = np.array(self.face_labels)

        # Encode labels
        y_encoded = self.label_encoder.fit_transform(y)

        # Split data for training and validation
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )

        # Train SVM model
        self.svm_model = SVC(
            kernel='rbf',
            probability=True,
            C=1.0,
            gamma='scale'
        )

        self.svm_model.fit(X_train, y_train)

        # Evaluate model
        y_pred = self.svm_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"Model training completed!")
        print(f"Validation Accuracy: {accuracy:.4f}")

        # Detailed classification report
        class_names = self.label_encoder.classes_
        report = classification_report(y_test, y_pred, target_names=class_names)
        print("Classification Report:")
        print(report)

        # Save model
        self.save_model()

        return accuracy

    def save_model(self):
        """Save trained model and encodings"""
        print("Saving model...")

        model_data = {
            'svm_model': self.svm_model,
            'label_encoder': self.label_encoder,
            'face_encodings': self.face_encodings,
            'face_labels': self.face_labels,
            'recognition_threshold': self.recognition_threshold,
            'confidence_threshold': self.confidence_threshold
        }

        with open(f"{self.model_path}/face_recognition_model.pkl", 'wb') as f:
            pickle.dump(model_data, f)

        print("Model saved successfully!")

    def load_model(self):
        """Load pre-trained model"""
        model_file = f"{self.model_path}/face_recognition_model.pkl"

        if not os.path.exists(model_file):
            print("No trained model found. Please train the model first.")
            return False

        print("Loading trained model...")

        with open(model_file, 'rb') as f:
            model_data = pickle.load(f)

        self.svm_model = model_data['svm_model']
        self.label_encoder = model_data['label_encoder']
        self.face_encodings = model_data['face_encodings']
        self.face_labels = model_data['face_labels']
        self.recognition_threshold = model_data.get('recognition_threshold', 0.6)
        self.confidence_threshold = model_data.get('confidence_threshold', 0.5)

        print("Model loaded successfully!")
        print(f"Known persons: {list(self.label_encoder.classes_)}")
        return True

    def recognize_face(self, face_encoding):
        """
        Recognize a face using the trained model
        """
        if self.svm_model is None:
            return "Unknown", 0.0

        # Reshape for prediction
        face_encoding = face_encoding.reshape(1, -1)

        # Get prediction probabilities
        probabilities = self.svm_model.predict_proba(face_encoding)[0]
        max_prob = np.max(probabilities)

        if max_prob > self.confidence_threshold:
            predicted_class = self.svm_model.predict(face_encoding)[0]
            person_name = self.label_encoder.inverse_transform([predicted_class])[0]
            return person_name, max_prob
        else:
            return "Unknown", max_prob

    def real_time_recognition(self):
        """
        Perform real-time face recognition using webcam
        """
        if self.svm_model is None:
            print("No trained model available. Please train the model first.")
            return

        print("Starting real-time face recognition...")
        print("Press 'q' to quit, 's' to save unknown face, 'r' to reset stats")

        cap = cv2.VideoCapture(0)

        # Performance tracking
        frame_count = 0
        fps_start_time = time.time()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Flip frame for natural interaction
            frame = cv2.flip(frame, 1)
            frame_count += 1

            # Convert BGR to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Find face locations and encodings
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            # Process each face
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Recognize face
                name, confidence = self.recognize_face(face_encoding)

                # Update statistics
                self.session_stats['total_detections'] += 1
                if name != "Unknown":
                    self.session_stats['successful_recognitions'] += 1
                    self.recognition_stats[name] += 1
                else:
                    self.session_stats['unknown_faces'] += 1

                # Choose color based on recognition
                if name != "Unknown":
                    color = (0, 255, 0)  # Green for known faces
                    label = f"{name} ({confidence:.2f})"
                else:
                    color = (0, 0, 255)  # Red for unknown faces
                    label = f"Unknown ({confidence:.2f})"

                # Draw rectangle and label
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
                cv2.putText(frame, label, (left + 6, bottom - 6),
                           cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)

            # Calculate and display FPS
            if frame_count % 30 == 0:
                fps = 30 / (time.time() - fps_start_time)
                fps_start_time = time.time()
            else:
                fps = 0

            if fps > 0:
                cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Display statistics
            cv2.putText(frame, f"Detections: {self.session_stats['total_detections']}", (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, f"Recognized: {self.session_stats['successful_recognitions']}", (10, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, f"Unknown: {self.session_stats['unknown_faces']}", (10, 100),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            cv2.imshow('Real-Time Face Recognition', frame)

            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s') and len(face_locations) > 0:
                # Save unknown face for later training
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                unknown_path = f"{self.data_path}/unknown/unknown_{timestamp}.jpg"
                cv2.imwrite(unknown_path, frame)
                print(f"Unknown face saved: {unknown_path}")
            elif key == ord('r'):
                # Reset statistics
                self.reset_session_stats()
                print("Statistics reset")

        cap.release()
        cv2.destroyAllWindows()

        # Display final statistics
        self.display_session_summary()

    def reset_session_stats(self):
        """Reset session statistics"""
        self.session_stats = {
            'total_detections': 0,
            'successful_recognitions': 0,
            'unknown_faces': 0,
            'start_time': datetime.now()
        }
        self.recognition_stats.clear()

    def display_session_summary(self):
        """Display session statistics summary"""
        print("\n" + "="*50)
        print("SESSION SUMMARY")
        print("="*50)

        duration = datetime.now() - self.session_stats['start_time']
        success_rate = 0
        if self.session_stats['total_detections'] > 0:
            success_rate = (self.session_stats['successful_recognitions'] /
                          self.session_stats['total_detections']) * 100

        print(f"Session Duration: {duration}")
        print(f"Total Face Detections: {self.session_stats['total_detections']}")
        print(f"Successful Recognitions: {self.session_stats['successful_recognitions']}")
        print(f"Unknown Faces: {self.session_stats['unknown_faces']}")
        print(f"Recognition Success Rate: {success_rate:.2f}%")

        if self.recognition_stats:
            print("\nRecognition Breakdown:")
            for person, count in sorted(self.recognition_stats.items()):
                print(f"  {person}: {count} times")

    def add_person_to_model(self, person_name, num_images=30):
        """
        Add a new person to existing model without retraining everything
        """
        print(f"Adding new person: {person_name}")

        # Capture new training data
        captured = self.capture_training_data(person_name, num_images)

        if captured > 0:
            # Extract encodings for new person
            person_dir = f"{self.data_path}/faces/{person_name}"
            new_encodings = []

            for image_file in os.listdir(person_dir):
                if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                image_path = os.path.join(person_dir, image_file)
                image = face_recognition.load_image_file(image_path)
                encodings = face_recognition.face_encodings(image)

                if encodings:
                    new_encodings.append(encodings[0])

            # Add to existing data
            self.face_encodings.extend(new_encodings)
            self.face_labels.extend([person_name] * len(new_encodings))

            # Retrain model
            self.train_model()

            print(f"Successfully added {person_name} to the model!")
        else:
            print(f"Failed to capture data for {person_name}")


def main():
    """
    Main function to demonstrate the face recognition system
    """
    print("="*60)
    print("REAL-TIME FACE RECOGNITION SYSTEM")
    print("="*60)

    # Initialize system
    face_system = FaceRecognitionSystem()

    while True:
        print("\nChoose an option:")
        print("1. Capture training data for new person")
        print("2. Train face recognition model")
        print("3. Load existing model")
        print("4. Start real-time recognition")
        print("5. Add person to existing model")
        print("6. View system statistics")
        print("7. Exit")

        choice = input("\nEnter your choice (1-7): ").strip()

        if choice == '1':
            person_name = input("Enter person's name: ").strip()
            if person_name:
                num_images = int(input("Number of images to capture (default 50): ") or "50")
                face_system.capture_training_data(person_name, num_images)
            else:
                print("Invalid name!")

        elif choice == '2':
            print("Extracting face encodings...")
            if face_system.extract_face_encodings() > 0:
                face_system.train_model()
            else:
                print("No training data found. Please capture training data first.")

        elif choice == '3':
            face_system.load_model()

        elif choice == '4':
            face_system.real_time_recognition()

        elif choice == '5':
            if face_system.svm_model is None:
                print("Please load or train a model first!")
            else:
                person_name = input("Enter new person's name: ").strip()
                if person_name:
                    face_system.add_person_to_model(person_name)
                else:
                    print("Invalid name!")

        elif choice == '6':
            face_system.display_session_summary()

        elif choice == '7':
            print("Goodbye!")
            break

        else:
            print("Invalid choice! Please try again.")


if __name__ == "__main__":
    print("Required libraries: pip install opencv-python face-recognition scikit-learn matplotlib seaborn")
    print("Make sure you have a working webcam connected!")
    print()
    main()